In [ ]:
import numpy as np
import os
import sys
import glob
import cv2
from matplotlib import pyplot as plt
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
from mygenerator import DataGenerator

In [ ]:
list_predict = list()

for file in glob.glob("./test/*.png"):
    list_predict.append(file)
    
lab_predict = np.loadtxt('./labels_ts')

labels = dict(zip(list_predict, lab_predict))

partition = {'predict': list_predict}

In [ ]:
# Parameters
params = {'dim': 61,
          'batch_size': 1,
          'n_classes': 2,
          'n_channels': 1,
          'shuffle': False}

predict_generator = DataGenerator(partition['predict'], labels,  **params)

In [ ]:
# load json and create model
json_file = open('model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model")

In [ ]:
predict = loaded_model.predict_generator(predict_generator,steps = len(list_predict))

In [ ]:
probabilites = list(predict)
print(len(probabilites))

In [ ]:
probs = [probabilites[el][0] for el in range(len(probabilites))]

In [ ]:
print(probs[:10])

In [ ]:
# Visualize the first image
im = probs[:8281]
im = np.asarray(im)
im = im.reshape((91,91))
plt.figure(figsize=[5,5])
plt.subplot(121) 
plt.imshow(im, cmap='gray')

In [ ]:
# Create output folder
import os
test_path =  './test_outIm'
os.mkdir(test_path);

In [ ]:
# Retrieve names of the images
j = 0
list_name = list()
# Set image paths
images_path = "./test_im"
for infile in glob.glob(images_path + '/*.tif'):
    print(infile)
    p, im_name = os.path.split(infile)
    file, ext = os.path.splitext(im_name)
    list_name.append(file)
    
    
for num, i in enumerate(range(0,len(probabilites), 8281)):
    print(j, i)
    im =  np.asarray(probs[j:i+8281])
    im = im.reshape((91,91))
    plt.imshow(im*255, cmap='gray')
    cv2.imwrite('./'+ test_path  + '/' + list_name[num] + '.png' , im*255)
    j = i+8281